In [1]:
from tqdm import tqdm
import sys
sys.path.insert(0, '/home/guinzburg/NLP/Data')
from data_parser import parse_queries, parse_documents
from LMIR import LMIR

# Data
Load the files containing queries and documents using 'parse_queries' and 'parse_documents'

In [2]:
# Get all queries
queries = parse_queries('/home/guinzburg/NLP/Data/CISI.QRY')
# Get all documents
documents = parse_documents('/home/guinzburg/NLP/Data/CISI.ALL')

# KL Processing
For each query, find the document with the highest KL rate

In [ ]:
# for each query, find the document with the highest KL rate and store it a dictionary item in a list, e.g. [{"<QUERY_ID>":"123", "KL Rate": 1.23, <"QUERY>":"abc", "<DOCUMENT_TITLE>":"abc", "<QUERY_TEXT>":"abc"}]
top_kl_items = []
for q_id, q in tqdm(queries.items(), desc="Processing Queries"):
    ir = LMIR()
    max_kl_rate = 0
    for _, d in documents.items():
        d_tit = d["title"] # get document title
        d_txt = d["body"] # get document text
        ir.AddDocText(d_tit, d_txt)
        # ir.test() ##### NOT SURE WHAT IT IS FOR!
        kl = ir.RankKL(q)
    # iterate on all kl items to look for the one with the highest rate
    for kl_item in kl:
        kl_item_rate = kl_item[1]
        if kl_item_rate > max_kl_rate:
            max_kl_rate = kl_item_rate
            max_kl_doc_tit = kl_item[0]
    # retrieve document top kl ID's
    for d_id, d in documents.items():
        if d["title"] == max_kl_doc_tit:
            max_kl_doc_id = d_id
    
    top_kl_items.append({"Query ID": q_id,
                            "Query": q,
                            "KL Rate": max_kl_rate,
                            "Document ID": max_kl_doc_id,
                            "Document Title": max_kl_doc_tit,
                            "Document Body": documents[max_kl_doc_id]['body']})

Processing Queries:   5%|▌         | 6/112 [03:06<51:47, 29.32s/it]  

# Results
Get the top 10 queries with the highest KL rate

In [ ]:
# Get Top N elements from '{top_kl_items}'
# Using sorted() + lambda
N = 10
top_queries = sorted(top_kl_items, key = lambda x: x["KL Rate"], reverse = True)[:N]